In [193]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from pprint import pprint
from splinter import Browser

In [162]:
news = 'https://mars.nasa.gov/news/'
img = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
tweets = 'https://twitter.com/marswxreport?lang=en'
facts = 'https://space-facts.com/mars/'
hemis = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [35]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [36]:
db = client.news_db
collection = db.items

# News Scraping

In [117]:
response = requests.get(news)

news_soup = BeautifulSoup(response.text, 'lxml')

title = news_soup.find('div', class_= 'content_title')
news_title = title.a.text.strip()
print(news_title)

p = news_soup.find('div', class_= 'rollover_description_inner')
news_p = p.text.strip()
print(news_p)

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries.


# Image Scraping

In [118]:
response = requests.get(img)

img_soup = BeautifulSoup(response.text, 'lxml')

i = img_soup.find('a', class_ = 'button fancybox')

featured_img_url = 'https://www.jpl.nasa.gov' + i.attrs['data-fancybox-href']
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14106_ip.jpg


# Tweet Scraping

In [224]:
response = requests.get(tweets)

tweet_soup = BeautifulSoup(response.text, 'lxml')
# t = tweet_soup.find('p', class_ = 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
t = tweet_soup.find_all('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
# t = tweet_soup.find_all('div', class_='js-tweet-text-container')

# t = tweet_soup.find_all('body')
for i in t:
    if 'InSight' in i.text:
        i.a.decompose()
        print(i.text)
# pprint(t)
# mars_weather = t.contents[0]
# print(mars_weather)

InSight sol 455 (2020-03-08) low -95.4ºC (-139.8ºF) high -13.0ºC (8.5ºF)
winds from the SSE at 6.0 m/s (13.5 mph) gusting to 20.7 m/s (46.2 mph)
pressure at 6.40 hPa
InSight sol 454 (2020-03-06) low -96.0ºC (-140.8ºF) high -12.7ºC (9.1ºF)
winds from the SSE at 5.8 m/s (12.9 mph) gusting to 25.2 m/s (56.4 mph)
pressure at 6.30 hPa
InSight sol 453 (2020-03-05) low -95.1ºC (-139.1ºF) high -10.8ºC (12.6ºF)
winds from the SSW at 6.0 m/s (13.3 mph) gusting to 21.4 m/s (47.9 mph)
pressure at 6.30 hPa
InSight sol 452 (2020-03-05) low -94.2ºC (-137.7ºF) high -9.2ºC (15.4ºF)
winds from the SSW at 6.5 m/s (14.5 mph) gusting to 19.6 m/s (43.9 mph)
pressure at 6.30 hPa
InSight sol 451 (2020-03-03) low -93.6ºC (-136.4ºF) high -10.3ºC (13.4ºF)
winds from the SSE at 6.3 m/s (14.2 mph) gusting to 19.9 m/s (44.5 mph)
pressure at 6.30 hPa
InSight sol 450 (2020-03-02) low -93.5ºC (-136.4ºF) high -10.4ºC (13.3ºF)
winds from the SSW at 5.5 m/s (12.4 mph) gusting to 20.6 m/s (46.1 mph)
pressure at 6.30 hPa
I

# Facts Scraping

In [176]:
facts_html = pd.read_html(facts)
facts_df = facts_html[0]
facts_df.columns = ['Description', 'Data']
facts_df.set_index('Description', inplace=True)
facts_df

,Data
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [187]:
html_table = facts_df.to_html()
html_table = html_table.replace('\n', '')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Data</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

# Hemisphere Scraping

In [203]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(hemis)

hemisphere_image_urls = []

for i in range(4):
    link = browser.links.find_by_partial_text('Hemisphere')[i]
    link.click()
    title = browser.find_by_css('.title').first.text
    url = browser.find_by_text('Sample').first['href']
    hemi_dict = {'title':title, 'url':url}
    hemisphere_image_urls.append(hemi_dict)
    browser.back()

In [210]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
